Tạo các features bậc 2 và tương quan

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

# Item đầu vào, thời gian, điểm cách biệt, rating player
X = np.array([[12, -5, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4]])

# Khởi tạo check xem các polynomial bậc 2 có cái nào dùng được không, không dùng bậc 3 vì chậm vcl =)))
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
X_poly = poly.fit_transform(X)

print("Original Features Shape:", X.shape)
print("Transformed Features Shape:", X_poly.shape)


Tạo thêm các biến có thể lấy từ playbyplay data

In [ ]:
#Rating của đội
team_avg_rating = X[:, 2:].mean(axis=1)
#Tiền xử lý PlaybyPlay Data tạo ra thêm biến hiệp đấu
#Normalize cách biệt điểm
normalized_uppoints = X[:, 1] / (X[:, 0] + 1e-5) 

Scale lại các feature vì SVM nhạy hơn Neural Net

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_poly)


Từ các features đã đưa vào, xem các feature nào có ít ảnh hưởng để loại ( Thực ra là làm cho có chứ ít features vcl )

In [ ]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

selector = VarianceThreshold(threshold=0.01)
X_selected = selector.fit_transform(X_scaled)
rfe = RFE(estimator=SVR(kernel="linear"), n_features_to_select=10)
X_rfe = rfe.fit_transform(X_selected, y)  #y là label hay target của tập training

Check lại các feature ( Để thấy nó có ảnh hưởng thôi chứ chắc cũng ổn hết ;-;)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_rfe, y) 
print("Feature Importances:", model.feature_importances_)


Nếu đủ time thì automate bọn này để lấy dữ liệu live tự vào tự cập nhật (Pipeline Oách xà lách hỏi từ ChatGPT nhưng mà đéo biết có phải dùng Spark với Hadoop hay không ._. )

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2)),
    ('scaler', StandardScaler()),
    ('variance', VarianceThreshold(threshold=0.01)),
    ('rfe', RFE(estimator=SVR(kernel="linear"), n_features_to_select=10))
])

X_transformed = pipeline.fit_transform(X, y)  # Fit and transform
